##### `If your files originate from a Thermo instrument:`

Copy all your *.raw files to data/raw directory and you can use the following script to generate a samples.tsv file automatically:

In [ ]:
!(cd data/raw && ls *.raw > filelist.txt)
import pandas as pd
header_list = ["sample_name"]
df=pd.read_csv("data/raw/filelist.txt", names=header_list, index_col= None)
df["sample_name"]=df["sample_name"].replace(".raw*", value="", regex=True)
df["comment"] = " "
df["MAPnumber"] = " "
df.to_csv("config/samples.tsv", sep="\t")
df

##### `If your files originate from another instrument:`

Copy all your already converted files *.mzML to data/mzML directory and you can use the following script to generate a samples.tsv file automatically:

In [ ]:
!(cd data/mzML && ls *.mzML > filelist.txt)
import pandas as pd
header_list = ["sample_name"]
df=pd.read_csv("data/mzML/filelist.txt", names=header_list, index_col= None)
df["sample_name"]=df["sample_name"].replace(".mzML*", value="", regex=True)
df["comment"] = " "
df["MAPnumber"] = " "
df.to_csv("config/samples.tsv", sep="\t")
df

##### `Create a GNPS metadata table:`
This is datafile-dependent so it is preferable to do it interactively through a Jupyter notebook

In [1]:
!(cd results/Interim/mzML && ls PCfeature*.mzML > filelist.txt)

In [2]:
import pandas as pd
import numpy as np 
import os 

path= "results/GNPSexport/"
isExist= os.path.exists(path)
if not isExist:
    os.mkdir(path)

import pandas as pd
header_list = ["filename"]
df=pd.read_csv("results/Interim/mzML/filelist.txt", names=header_list, index_col= None)
df["filename"]=df["filename"].replace(to_replace= r'MDNAWGS', value= 'MDNA_WGS_', regex= True)
df['ATTRIBUTE_MAPID'] = np.arange(len(df))
df["ATTRIBUTE_MAPID"]= "MAP" + df["ATTRIBUTE_MAPID"].astype(str)
df['ATTRIBUTE_genomeID']=df['filename'].str.extract(r'(NBC_?\d*)')
df['ATTRIBUTE_genomeIDMDNA']=df['filename'].str.extract(r'(MDNAWGS?\d*|MDNA_WGS_?\d*)')
df['ATTRIBUTE_genomeID']=df['ATTRIBUTE_genomeID'].fillna(df['ATTRIBUTE_genomeIDMDNA'])
df["ATTRIBUTE_media"]= df['filename'].str.extract(r'(ISP2|DNPM|FPY12|MA|soyM\d*)')
df["ATTRIBUTE_comment"]= df['ATTRIBUTE_genomeID'].astype(str) +"_" + df["ATTRIBUTE_media"].astype(str)
df=df.drop(columns="ATTRIBUTE_genomeIDMDNA")
df.to_csv("results/GNPSexport/metadata.tsv", sep='\t')
df

,filename,ATTRIBUTE_MAPID,ATTRIBUTE_genomeID,ATTRIBUTE_media,ATTRIBUTE_comment
0,PCfeature_20210607_UMETAB211_POS_00029_DNPM_HL...,MAP0,NaN,DNPM,nan_DNPM
1,PCfeature_20210607_UMETAB211_POS_00029_MA_HLB7...,MAP1,NaN,MA,nan_MA
2,PCfeature_20210607_UMETAB211_POS_00105_ISP2_HL...,MAP2,NaN,ISP2,nan_ISP2
3,PCfeature_20210607_UMETAB211_POS_00105_MA_HLB7...,MAP3,NaN,MA,nan_MA
4,PCfeature_20210607_UMETAB211_POS_00105_soyM_HL...,MAP4,NaN,soyM,nan_soyM
5,PCfeature_20210608_UMETAB211_POS_00207_DNPM_HL...,MAP5,NaN,DNPM,nan_DNPM
6,PCfeature_20210608_UMETAB211_POS_00207_MA_HLB7...,MAP6,NaN,MA,nan_MA
7,PCfeature_20210609_UMETAB211_POS_00029_ISP2_HL...,MAP7,NaN,ISP2,nan_ISP2
8,PCfeature_20210609_UMETAB211_POS_00029_MA_HLB1...,MAP8,NaN,MA,nan_MA
9,PCfeature_20210609_UMETAB211_POS_00029_soyM_HL...,MAP9,NaN,soyM,nan_soyM


In [3]:
!(cd results/Interim/mzML && rm filelist.txt)